In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import lastfmUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from artists import artists
from artistsLM import artistsLM
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-02-27 15:51:44.897946


In [5]:
%load_ext autoreload
%autoreload
disc   = discogs('lastfm')
dutils = lastfmUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available


# Artists From Search

In [ ]:
192   Astronaut Ape                           set()
789   Color me Badd                           set()
1086  Dr.Alban                                set()
1087  Dyango                                  set()
1232  Elba Ramalho                            set()
1389  Fabolous and Jadakiss                   set()
1390  Frankie Valli & The Four Seasons        {'Frankie Valli & The Romans'}
1554  Gary Lewis                              {'Gary Lucas'}
1663  Heart                                   set()
1936  Jaded Heart                             set()
2509  Marco Masini                            {'Micro Mini'}
2510  Michael Franti & Spearhead              set()
2722  Oresund Space Collective                set()
3481  Social Distortion                       set()
3711  Tom Keifer                              set()
4258  The Whispers                            {'The Whites'}

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs('lastfm')
arts = artists(disc)
#arts.searchDiscogForArtist("The Whispers")

In [13]:
from webUtils import getWebData
url="https://www.last.fm/music/Coldplay/+albums"
url="https://www.last.fm/music/Coldplay/+albums?order=release_date"
savename="lastfm.p"
getWebData(base=url, savename=savename, useSafari=False)

  --> This file is 28.1kB.
lastfm.p size -> 28 kB


True

In [14]:
bsdata = getHTML("lastfm.p")

In [18]:
ols = bsdata.findAll("ol", {"class": "buffer-standard resource-list--release-list resource-list--release-list--with-20"})
for ol in ols:
    lis = ol.findAll("li", {"class": "resource-list--release-list-item-wrap"})
    for li in lis:
        
    print(len(lis))

20


In [15]:
bsdata

<!DOCTYPE html>
<html class="no-js playbar-masthead-release-shim youtube-provider-not-ready" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UwYPV15QGwYFXFlXDgU=",licenseKey:"0ed0ce50b0",applicationID:"5588594"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(21),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

In [ ]:
"https://www.last.fm/music/Coldplay/+albums?order=release_date"

In [ ]:
https://www.last.fm/music/Coldplay/+albums

In [ ]:
def getURLData(url, debug=True):    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading: {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
    
    return data, response

In [ ]:
myMusicMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [ ]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

In [ ]:
from json import loads
baseURL = "https://www.allmusic.com/search/artists/"
for i,(artistName,artistVals) in enumerate(myMusicMap.items()):
    if i < 3871:
        continue
    print(i,'/',len(myMusicMap),'  \t',artistName)
    url     = urllib.parse.urljoin(baseURL, quote(artistName))

    data, response = getURLData(url)
    if response.code == 200:
        bsdata = getHTML(data)
    else:
        continue
        
    infos = bsdata.findAll("div", {"class": "info"})
    for info in infos:
        for name in info.findAll("div", {"class": "name"}):
            ref = name.find("a")
            if ref is None:
                continue
            href     = ref.attrs['href']
            artistID = loads(ref.attrs['data-tooltip'])['id']
            name     = ref.text
            if allMusicIDMap.get(artistID) is None:
                allMusicIDMap[artistID] = [name, href]
                print("\t{0: <70}{1: <20}".format(name,artistID))
                
    print("  -->",len(allMusicIDMap))
    sleep(2)
    if i % 50 == 0:
        savename = "collections/allMusicIDMap.p"
        print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
        saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

In [ ]:

savename = "collections/allMusicIDMap.p"
print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

In [ ]:
allMusicIDMap['MN0000961234']

In [ ]:
amInfo  = allMusicIDMap['MN0000961234']
url     = "{0}/discography/all".format(amInfo[1])
data, response = getURLData(url, debug=True)

In [ ]:
savename = "MN0000961234.p"
force = True
print("Saving {0} (force={1})".format(savename, force))
saveJoblib(data=data, filename=savename, compress=True)

In [ ]:
data = getFile(savename)

In [ ]:
url = "https://www.allmusic.com/artist/dave-matthews-band-mn0000961234/discography/all"

# All Music

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [ ]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

In [ ]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
allMusicIDMap['MN0000986327']

In [ ]:
from time import sleep
for i,(artistID,artistVals) in enumerate(allMusicIDMap.items()):
    if i <= 49496:
        continue
    artistName,ref = artistVals
    print(i,'/',len(allMusicIDMap),'\t',artistName,'\t',ref)
    url      = artsAM.getArtistURL(ref)
    savename = artsAM.getArtistSavename(artistID)
    artsAM.downloadArtistURL(url, savename, force=True)

In [ ]:
data = artsAM.getData('/Users/tgadfort/Music/Discog/artists-allmusic/62/0000567862.p')

In [ ]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
file = "MN0000961234.p"
file = "/Users/tgadfort/Music/Discog/artists-allmusic/27/0000986327.p"
artAM = artistAM()
data = artAM.getData(file)

In [ ]:
data

In [ ]:
print("Artist")
info = data.artist.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("URL")
info = data.url.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("ID")
info = data.ID.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Pages")
info = data.pages.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Profile")
info = data.profile.get()
for k,v in info.items():
    if k == "profile":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)
    elif k == "members":
        print("\t",k)
        for v2 in v:
            print("\t\t",v2)
    else:
        print('\t',k,'\t',v)
            

print("Media")
info = data.media.get()
for k,v in info.items():
    if k == "media":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)


print("MediaCounts")
info = data.mediaCounts.get()
for k,v in info.items():
    print("\t",k,'\t',v)